In [ ]:
%load_ext autoreload
%autoreload 2

import geoviews as gv
import geoviews.feature as gf
from cartopy import crs as ccrs

import paleoviews as pv
from gprm.datasets import Reconstructions

gv.extension('bokeh', 'matplotlib')



In [ ]:
M2021 = Reconstructions.fetch_Merdith2021()


In [ ]:
Continents = M2021.polygon_snapshot('continents', 250)


In [ ]:
dim = 300
central_longitude = 0.

gv.Polygons(pv.wrap_reconstructed_features(Continents.reconstructed_polygons, central_longitude=central_longitude), 
            vdims=['NAME', 'PLATEID1', 'FROMAGE']).opts(cmap='inferno_r',
    tools=['hover'], height=dim, width=int(dim*1.9), projection=ccrs.Robinson(central_longitude=central_longitude), color_index='FROMAGE'
)

In [ ]:
Plates = M2021.plate_snapshot(250.)

central_longitude = 90.

features = [resolved_topology.get_resolved_feature() for resolved_topology in Plates.resolved_topologies]
dim = 300
gv.Polygons(pv.wrap_features(features), 
            vdims=['NAME', 'PLATEID1']).opts(
    tools=['hover'], height=dim, width=int(dim*1.9), projection=ccrs.Robinson(central_longitude=central_longitude), fill_color='red'
)

In [ ]:
Plates = M2021.plate_snapshot(150.)
Continents = M2021.polygon_snapshot('continents', 150)
features = [resolved_topology.get_resolved_feature() for resolved_topology in Plates.resolved_topologies]

dim = 600
central_longitude = 0.


gv.Polygons(pv.wrap_features(features), 
            vdims=['NAME', 'PLATEID1']).opts(
    height=dim, width=int(dim*1.9), projection=ccrs.Robinson(central_longitude=central_longitude), fill_color='red'
) * gv.Polygons(pv.wrap_reconstructed_features(Continents.reconstructed_polygons, central_longitude=central_longitude), 
            vdims=['NAME', 'PLATEID1', 'FROMAGE']).opts(cmap='inferno_r',
    tools=['hover'], height=dim, width=int(dim*1.9), projection=ccrs.Robinson(central_longitude=central_longitude), color_index='FROMAGE'
)


In [ ]:
M2016 = Reconstructions.fetch_Muller2016()

Plates = M2016.plate_snapshot(0.)

central_longitude = 90.

#features = [resolved_topology.get_resolved_feature() for resolved_topology in Plates.resolved_topological_sections]
features = Plates.get_boundary_features()
dim = 300
gv.Path(pv.wrap_features(features), 
            vdims=['NAME', 'PLATEID1']).opts(
    tools=['hover'], height=dim, width=int(dim*1.9), projection=ccrs.Robinson(central_longitude=central_longitude),
)


In [ ]:
import numpy as np

Plates = M2016.plate_snapshot(0., anchor_plate_id=0)
velocities = Plates.velocity_field()

azimuths = np.radians(90.-np.degrees(+np.array(velocities.velocity_azimuth)))
magnitudes = np.array(velocities.velocity_magnitude)

vectorfield = gv.VectorField((velocities.longitude, velocities.latitude, 
                              azimuths,
                              magnitudes*0.04))

dim = 600
gv.Polygons(pv.wrap_features([resolved_topology.get_resolved_feature() for resolved_topology in Plates.resolved_topologies]), 
            vdims=['NAME', 'PLATEID1']).opts(cmap='gray',
    tools=['hover'], height=dim, width=int(dim*1.9), projection=ccrs.Robinson(),
) * gf.coastline * vectorfield.opts(
    magnitude='Magnitude', color='Magnitude', cmap='fire', width=900, height=500,
    projection=ccrs.Robinson(), global_extent=True, normalize_lengths=False)


In [ ]:
#TODO platetree


gv.help(gv.Points)

In [ ]:
from gprm.datasets import Rocks

OD = Rocks.BaseMetalDeposits('PbZn-CD')



In [ ]:
reconstruction_time = 60.
Plates = M2021.plate_snapshot(reconstruction_time)
Continents = M2021.polygon_snapshot('continents', reconstruction_time)
ODr = M2021.reconstruct(M2021.assign_plate_ids(OD), reconstruction_time)
ODr.Longitude = ODr.geometry.x
ODr.Latitude = ODr.geometry.y

dim = 600

gv.Polygons(pv.wrap_features([resolved_topology.get_resolved_feature() for resolved_topology in Plates.resolved_topologies]), vdims=['NAME', 'PLATEID1']).opts(
    height=dim, width=int(dim*1.9), projection=ccrs.Robinson(), fill_color='red'
) * gv.Polygons(pv.wrap_reconstructed_features(Continents.reconstructed_polygons), vdims=['NAME', 'PLATEID1', 'FROMAGE']).opts(
    cmap='inferno_r', height=dim, width=int(dim*1.9), projection=ccrs.Robinson(), color_index='FROMAGE'
) * gv.Points(ODr, vdims=['Deposit', 'Age Type']).opts(
    tools=['hover'], height=dim, width=int(dim*1.9), projection=ccrs.Robinson(), color='red', size=5,
)